In [11]:
import pandas as pd
import nltk, re, string, collections
from nltk.util import ngrams 

In [1]:
path = 'база данных.csv'

In [4]:
data = pd.read_csv(path)

In [6]:
data.head()

,Название,Местоположение,Часовой пояс,Улица,Примечание,Адрес,На карте,Описание,Адрес сайта,Адрес электронной почты,...,Types[],Start,ID места ЕИПСК,ServiceName,ЕИПСК,Культура.РФ,Госкаталог,Статистика,Дата создания записи,Дата последнего обновления записи
0,Аксёновская сельская библиотека-филиал № 4,Галичский район,Europe/Moscow,"д. Аксёново, 24",NaN,NaN,"{""coordinates"":[42.50301500000001,58.309780999...","<p style=""text-align:left""><span>Библиотека ос...",NaN,biblioteka-gorkogo.galich@yandex.ru,...,NaN,NaN,25431,"[{""serviceName"":""Культура.рф""}]",NaN,NaN,NaN,NaN,2016-10-20T07:48:12Z,2016-10-24T14:07:12Z
1,Мелёшинская сельская библиотека-филиал № 15,Галичский район,Europe/Moscow,"д. Мелёшино, ул. Центральная, 16",NaN,NaN,"{""coordinates"":[42.54122,58.438008],""type"":""Po...","<p style=""text-align:left""><span> Библиотека о...",NaN,biblioteka-gorkogo.galich@yandex.ru,...,NaN,NaN,25427,"[{""serviceName"":""Культура.рф""},{""serviceName"":...",NaN,NaN,NaN,NaN,2016-10-20T06:05:30Z,2016-10-24T13:57:48Z
2,Ряжская центральная библиотека,Ряжский район,Europe/Moscow,"г. Ряжск, пл. Советская, 11",NaN,NaN,"{""coordinates"":[40.06328100000002,53.708139000...","<p style=""text-align:left""><span>В феврале 186...",http://www.ryajsk-lib.ru,ryagsk-biblioteka2010@yandex.ru,...,NaN,NaN,25410,"[{""serviceName"":""Культура.рф""}]",NaN,NaN,NaN,NaN,2016-10-19T12:24:27Z,2016-10-24T13:17:32Z
3,Левковская сельская библиотека-филиал № 12,Галичский район,Europe/Moscow,"д. Левково, ул. Молодёжная, 1",NaN,NaN,"{""coordinates"":[42.187105,58.552082999999996],...","<p style=""text-align:left""><span>Левковская се...",NaN,biblioteka-gorkogo.galich@yandex.ru,...,NaN,NaN,25388,"[{""serviceName"":""Культура.рф""}]",NaN,NaN,NaN,NaN,2016-10-19T07:51:11Z,2016-10-24T12:54:32Z
4,Сасовская центральная библиотека,Сасово,Europe/Moscow,"ул. Ленина, 40",NaN,NaN,"{""coordinates"":[41.91434900000001,54.341094999...","<p style=""text-align:left""><span><span>Сасовск...",NaN,foliant@pochtamt.ru,...,NaN,NaN,25379,"[{""serviceName"":""Культура.рф""}]",NaN,NaN,NaN,NaN,2016-10-18T14:47:35Z,2016-10-24T12:44:13Z


In [9]:
data.keys()

Index(['Название', 'Местоположение', 'Часовой пояс', 'Улица', 'Примечание',
       'Адрес', 'На карте', 'Описание', 'Адрес сайта',
       'Адрес электронной почты', 'Номер', 'Изображение',
       'Дополнительные изображения', 'Тип категории', 'Категория',
       'Юридическое лицо', 'Местоположение.1', 'Часовой пояс.1', 'Улица.1',
       'Примечание.1', 'Адрес.1', 'На карте.1', 'ИНН', 'Принадлежность',
       'Подчинение', 'Timezone', 'Название.1', 'Идентификатор ЕИПСК', 'Тэги',
       'Hosting', 'ПН', 'ВТ', 'СР', 'ЧТ', 'ПТ', 'СБ', 'ВС',
       'Категория учреждения', 'Аудитория', 'Язык', 'Профессиональный уровень',
       'Виртуальный тур', 'Types[]', 'Start', 'ID места ЕИПСК', 'ServiceName',
       'ЕИПСК', 'Культура.РФ', 'Госкаталог', 'Статистика',
       'Дата создания записи', 'Дата последнего обновления записи'],
      dtype='object')

# Описания

In [8]:
with open('descriptions.txt', 'w', encoding='utf-8') as f:
    for i in data['Описание']:
        f.write(i + '\n')

### Bigrams

In [42]:
string.punctuation += '–'

In [103]:
def text_cleaning(text):
    text = re.sub('<.*?>|\xad','',text)
    text = re.sub("[" + string.punctuation + "]", " ", text)
    text = re.sub('\s+', ' ', text).strip()
    return text.lower()

def word_tok(text):
    return text.split()


all_descrip = ' '.join(data['Описание'])
all_descrip1 = text_cleaning(all_descrip)
words_desc = word_tok(all_descrip1)

In [104]:
all_descrip1[:300]

'библиотека основана в 1973 году расположена в деревне аксёново дмитриевского сельского поселения галичского муниципального района в зоне обслуживания библиотеки 100 человек читателей по учреждению 91 книжный фонд составляет 3306 экз книг библиотекарем работает поздина олеся олеговна библиотека тесно'

In [86]:
words_desc[:10]

['библиотека',
 'основана',
 'в',
 '1973',
 'году',
 'расположена',
 'в',
 'деревне',
 'аксёново',
 'дмитриевского']

In [87]:
def top_ngrams(text, num_n, top, all_=False):
    
    grams = ngrams(text, num_n)
    gram_freq = collections.Counter(grams)
    
    if all_: return gram_freq
    return gram_freq.most_common(top)

In [88]:
bigrams = top_ngrams(words_desc, 2, 100)

In [89]:
bigrams[:10]

[(('в', 'библиотеке'), 4205),
 (('фонд', 'библиотеки'), 3226),
 (('книжный', 'фонд'), 2769),
 (('сельская', 'библиотека'), 2256),
 (('библиотеки', 'составляет'), 1785),
 (('детская', 'библиотека'), 1427),
 (('году', 'в'), 1400),
 (('настоящее', 'время'), 1345),
 (('для', 'детей'), 1332),
 (('библиотека', 'была'), 1223)]

In [90]:
trigrams = top_ngrams(words_desc, 3, 100)

In [91]:
trigrams[:10]

[(('фонд', 'библиотеки', 'составляет'), 1531),
 (('в', 'настоящее', 'время'), 1154),
 (('книжный', 'фонд', 'библиотеки'), 704),
 (('книжный', 'фонд', 'составляет'), 629),
 (('была', 'открыта', 'в'), 627),
 (('библиотека', 'филиал', '№'), 624),
 (('на', 'сегодняшний', 'день'), 544),
 (('фонд', 'библиотеки', 'насчитывает'), 517),
 (('централизованной', 'библиотечной', 'системы'), 507),
 (('в', 'библиотеке', 'работает'), 469)]

 ##### Что можно достать?

* <b>Дата основания</b> - ('была', 'открыта'), ('библиотека', 'основана'),  
* <b>Размер фонда</b> - ('фонд', 'насчитывает'), ('базе', 'библиотеки'), ('насчитывает', 'более'), ('библиотечный', 'фонд'), ('тысяч', 'экземпляров'), ('библиотеки', 'насчитывает'), ('фонд', 'библиотеки'), ('книжный', 'фонд')
* <b>Кол-во читателей</b> - ('количество', 'читателей')
* <b>Зона обслуживания</b> - ('библиотека', 'обслуживает')
* <b>Девиз библиотеки</b> (нужно ли?)
* <b>Чем является ?</b> - ('библиотека', 'является')
* <b>Что есть в библиотеке</b> - ('в', 'библиотеке', 'работает')
* <b>Где находится (возможно, это не просто адрес, а что-то про здание)</b> - ('библиотека', 'находится', 'в')
* <b>Какие есть активности</b> - ('клубы', 'по', 'интересам'), ('на', 'базе', 'библиотеки')
* <b>Информация о наличии архива</b> - ('централизованная', 'библиотечная', 'система')

Примеры: 
    
('фонд', 'библиотеки', 'составляет')
('книжный', 'фонд', 'библиотеки')
('была', 'открыта', 'в')
('книжный', 'фонд', 'составляет')
('фонд', 'библиотеки', 'насчитывает')
('в', 'библиотеке', 'работает')
('библиотека', 'была', 'открыта')
('библиотека', 'находится', 'в')
('на', 'базе', 'библиотеки')
('библиотека', 'основана', 'в')
('входит', 'в', 'состав')
('библиотека', 'переехала', 'в')
('клубы', 'по', 'интересам')
('была', 'основана', 'в')
('года', 'в', 'библиотеке')
('находится', 'в', 'здании')
('в', 'библиотеке', 'работают')
('централизованная', 'библиотечная', 'система')
('экземпляров', 'книг', 'и')
('услугами', 'библиотеки', 'пользуются')
('библиотека', 'была', 'основана')
('фонд', 'составляет', 'более')
('при', 'библиотеке', 'работает')
('000', 'экземпляров', 'книг')
('библиотечный', 'фонд', 'составляет')
('централизованная', 'библиотечная', 'система»')
('ежегодно', 'библиотека', 'обслуживает')
('библиотеки', 'насчитывает', 'более')
('тысяч', 'экземпляров', 'книг')
('году', 'книжный', 'фонд')
('читателями', 'библиотеки', 'являются')
('работают', 'клубы', 'по')

In [105]:
with open('decript_clean.txt', 'w', encoding='utf-8') as f:
    all_descrip2 = re.sub('<.*?>|\xad', '', all_descrip)
    all_descrip2 = re.sub('\s+', ' ', all_descrip2).strip()
    f.write(all_descrip2)

## Word2vec для поиска синонимов

In [32]:
import gensim
from gensim.models import Word2Vec, KeyedVectors
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')

russian_stopwords = set(stopwords.words('russian'))

In [52]:
from gensim.test.utils import datapath

wv_from_bin = KeyedVectors.load_word2vec_format(datapath("182/model.bin"), binary=True)

In [63]:
wv_from_bin.similar_by_vector(wv_from_bin.wv['мама_NOUN'], topn=5)

[('мама_NOUN', 1.0),
 ('бабушка_NOUN', 0.8300854563713074),
 ('мама_PROPN', 0.8136544227600098),
 ('мать_NOUN', 0.7809500694274902),
 ('тетя_NOUN', 0.7670087218284607)]

In [74]:
wv_from_bin.similar_by_vector(wv_from_bin.wv['учредить_VERB'], topn=10)

[('учредить_VERB', 1.0),
 ('учреждать_VERB', 0.7878089547157288),
 ('учреждёть_ADJ', 0.6747399568557739),
 ('учреждать_ADJ', 0.6426123976707458),
 ('учредить_ADJ', 0.6390299797058105),
 ('учредить_NOUN', 0.6258695125579834),
 ('учредить_PROPN', 0.6163011193275452),
 ('учреждать_PROPN', 0.6056380271911621),
 ('учреждёть_VERB', 0.568996787071228),
 ('учреждённый_ADJ', 0.5525091290473938)]

In [71]:
wv_from_bin.similar_by_vector(wv_from_bin.wv['основать_VERB'], topn=10)

[('основать_VERB', 1.0),
 ('основывать_VERB', 0.7264178395271301),
 ('основавший_VERB', 0.6287158727645874),
 ('основать_ADJ', 0.6052384376525879),
 ('основывать_ADJ', 0.5743699073791504),
 ('основатель_NOUN', 0.5615928173065186),
 ('основатель_PROPN', 0.5217398405075073),
 ('обосноваться_VERB', 0.5102220773696899),
 ('учредить_VERB', 0.48151713609695435),
 ('основательница_NOUN', 0.47178083658218384)]

In [75]:
wv_from_bin.similar_by_vector(wv_from_bin.wv['открывать_VERB'], topn=10)

[('открывать_VERB', 1.0),
 ('открыть_VERB', 0.8137675523757935),
 ('открываться_VERB', 0.7574870586395264),
 ('закрывать_VERB', 0.6416383981704712),
 ('открытый_VERB', 0.6374810934066772),
 ('раскрывать_VERB', 0.6209543943405151),
 ('открывать_ADJ', 0.607235312461853),
 ('открытие_NOUN', 0.6052603125572205),
 ('приоткрывать_VERB', 0.5909624695777893),
 ('открыться_VERB', 0.5885521173477173)]